Get the dataset images

In [1]:
!git clone https://github.com/LeoEmmanuelJoint/FAA_project.git

Cloning into 'FAA_project'...
remote: Enumerating objects: 24403, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 24403 (delta 2), reused 7 (delta 2), pack-reused 24396
Receiving objects: 100% (24403/24403), 41.03 MiB | 29.53 MiB/s, done.
Resolving deltas: 100% (3/3), done.


Setting the train and test dataset

Creating the dataset

In [2]:
import os
import cv2

def create_training_data(folder_path, classes, img_size):
    X_train = []
    y_train = []
    for category in classes:
        path = os.path.join(folder_path, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                # new_array = cv2.resize(img_array, (img_size, img_size))
                X_train.append(img_array)
                y_train.append(class_num)
            except Exception as e:
                pass
    print("Converted image shape: %d" + str(X_train[0].shape))
    return(X_train, y_train)

In [3]:
import numpy as np

train_data_path = "/kaggle/working/FAA_project/data"
classes = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]


X_train, y_train = create_training_data(train_data_path, classes, 48)

X_train = np.array(X_train)
y_train = np.array(y_train)

Converted image shape: %d(48, 48, 3)


In [4]:
X_test = None
y_test = None
epochs = 30
batch_size = 8

number_of_labels = len(classes)
input_shape = X_train.shape
print(input_shape)

(25409, 48, 48, 3)


Building the model based on the architecture :
https://www.nature.com/articles/s41598-022-11173-0

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D

model=Sequential()
# Feature extraction layers
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
# Classification layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(number_of_labels, activation='softmax'))

model.summary()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 44, 44, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        3

Compiling

In [6]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy'])
# #categorical_crossentropy shoyld have been used, yet it does not work

In [7]:
print(y_train.shape)
print(X_train.shape)

(25409,)
(25409, 48, 48, 3)


Training

In [8]:
from time import time
import datetime

start_time = time()

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True)

end_time = time()
duration = end_time-start_time

print("Training duration: " + str(datetime.timedelta(seconds=duration)))

Epoch 1/30


2023-06-18 07:15:36.322396: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


3177/3177 [==============================] - 24s 5ms/step - loss: 1.5872 - accuracy: 0.3478
Epoch 2/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.4166 - accuracy: 0.4387
Epoch 3/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.3025 - accuracy: 0.4835
Epoch 4/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.2291 - accuracy: 0.5212
Epoch 5/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.1565 - accuracy: 0.5482
Epoch 6/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.0963 - accuracy: 0.5749
Epoch 7/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.0556 - accuracy: 0.5917
Epoch 8/30
3177/3177 [==============================] - 17s 5ms/step - loss: 1.0210 - accuracy: 0.6084
Epoch 9/30
3177/3177 [==============================] - 17s 5ms/step - loss: 0.9854 - accuracy: 0.6224
Epoch 10/30
3177/3177 [==============================] - 16s 5ms/step - loss: 0.9547

Test

In [9]:
#from sklearn.metrics import classification_report

#print("TEST RESULTS")
#y_test_model = model.predict(X_test)
#print(classification_report(y_test, y_test_model))

Save the model

In [10]:
model_json = model.to_json()
with open("model_convnet.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_convnet.h5")
print("Model saved")

Model saved
